Credit to @ulrich07 - I simply took his notebook and filtered out all but 2021 player targets and then averaged for each player.

Here is his noteboook that I forked.

https://www.kaggle.com/ulrich07/baseline-model-player-mean-or-median

In [ ]:
import pandas as pd
import os
import gc
from tqdm import tqdm

import mlb
env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

In [ ]:
ROOT_DIR = "../input/mlb-player-digital-engagement-forecasting"

In [ ]:
tr = pd.read_csv(f"{ROOT_DIR}/train.csv")
print(tr.shape)
gc.collect()

## COMPUTING PLAYER MEAN

In [ ]:
N_DATES = tr.shape[0]
d = []
for idx in tqdm(range(N_DATES)):
    u = eval(tr.iloc[idx, 1])
    d += u
#================

In [ ]:
tgt_df = pd.DataFrame(d)
print(tgt_df.shape)

In [ ]:
tgt_df['year'] = pd.DatetimeIndex(tgt_df['engagementMetricsDate']).year
tgt_df['month'] = pd.DatetimeIndex(tgt_df['engagementMetricsDate']).month

In [ ]:
tgt_df

In [ ]:
new_df = tgt_df[tgt_df['year'] == 2021]
new_df = new_df[tgt_df['month'] >= 4]
new_df.head()

In [ ]:
### group by year?
player_avg = new_df.groupby(["playerId"])[["target1","target2","target3","target4"]].mean().reset_index()
player_median = new_df.groupby(["playerId"])[["target1","target2","target3","target4"]].median().reset_index()
player_mean=pd.concat([player_avg, player_median], axis=1).groupby(axis=1, level=0).mean()
player_mean["target1"] = .85 * player_mean["target1"]
player_mean["target2"] = .85 * player_mean["target2"]
player_mean["target3"] = .85 * player_mean["target3"]
player_mean["target4"] = .85 * player_mean["target4"]
gc.collect()
print(player_mean.shape)
player_mean

In [ ]:
def process_pred(df):
    df["playerId"] = df["date_playerId"].apply(lambda x: int( x.split("_")[1] ) )
    df.drop(["target1","target2","target3","target4"], axis=1, inplace=True)
    df = df.merge(player_mean, on="playerId", how="left")
    df.drop("playerId", axis=1, inplace=True)
    df = df.fillna(0.)
    return df
#===================

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df = process_pred(sample_prediction_df)
    env.predict(sample_prediction_df)

In [ ]:
sample_prediction_df.head()